# MOWNIT
## Laboratorium 2
Jakub Karbowski

In [1]:
using Plots
using Interact
using DataFrames

WebIO._IJuliaInit()

# Lagrange
$$
\begin{align}
\text{Dane:}&\quad(x_1,y_1),\dots,(x_n,y_n) \\
L_k(x) &= \prod_{i=1,i\ne k}^n \frac{x-x_i}{x_k-x_i} \\
P_n(x) &= \sum_{k=1}^n y_k L_k(x)
\end{align}
$$

In [2]:
function lagrange(points)
    n = length(points)
    xs = getindex.(points, 1)
    ys = getindex.(points, 2)
    
    L(k, x) = prod([ (x - xs[i]) / (xs[k] - xs[i])
              for i = 1:n if i != k ])
    
    P(x) = sum([ ys[k] * L(k, x)
           for k = 1:n])
end

lagrange (generic function with 1 method)

# Newton
Wzorki z wykładu. Buduję macierz.

In [3]:
function newton(points)
    n = length(points)
    xs = getindex.(points, 1)
    ys = getindex.(points, 2)
    
    A = [ys zeros(n, n-1)]
    
    for j = 2:n, i = j:n
        A[i,j] = (A[i,j-1] - A[i-1,j-1]) /
              (xs[i] - xs[1 + (i-1) - (j-1)])
    end
    
    P(x) = A[1,1] + sum([ A[k,k] * prod([ (x - xs[i])
                                   for i = 1:k-1 ])
                    for k = 2:n ])
end

newton (generic function with 1 method)

# Węzły Czebyszewa
Dla przedziału $[-1,1]$:
$$
\begin{align}
x_k = \cos\left(\frac{2k-1}{2n}\pi\right)\text{,}\quad k=1,\dots,n
\end{align}
$$

Dla przedziału $[a,b]$:
$$
\begin{align}
x_k = \frac{a+b}{2} + \frac{b-a}{2} \cos\left(\frac{2k-1}{2n}\pi\right)\text{,}\quad k=1,\dots,n
\end{align}
$$

In [4]:
czeb(k, n) = cos((2k-1)/2n * π)
czeb(k, n, a, b) = (a+b)/2 + (b-a)/2 * czeb(k, n)

czeb (generic function with 2 methods)

# Błąd

In [5]:
maxerr(f, g, x) = maximum(@. abs(f(x) - g(x)) )
sqerr(f, g, x) = sum(@. ( f(x) - g(x) )^2 )

sqerr (generic function with 1 method)

# Moja funkcja

In [6]:
k = 2
m = 1
f(x) = exp(-k*sin(m*x))+k*cos(m*x)
xlo = -3pi
xhi = 3pi
xs = range(xlo, xhi, length=1000)

-9.42477796076938:0.018868424345884642:9.42477796076938

# Visualization
Czasami nie chce się aktualizować
po pierwszym włączeniu.

In [7]:
@manipulate for method=[lagrange, newton], spacing=[:even, :czeb], n=3:50
    nodes = if spacing == :czeb
                czeb.(1:n, n, xlo, xhi)
            else
                range(xlo, xhi, length=n)
            end
    
    P = method(zip(nodes, f.(nodes)))
    
    emax = maxerr(f, P, xs)
    esq  =  sqerr(f, P, xs)

    # just plots
    
    ylo = minimum(f.(xs))
    yhi = maximum(f.(xs))
    ycenter = (ylo + yhi) / 2
    xcenter = (xlo + xhi) / 2
    
    plot(
        xs,
        f.(xs),
        xlims=(xcenter + 1.1(xlo - xcenter), xcenter + 1.1(xhi - xcenter)),
#         ylims=(ycenter + 1.2(ylo - ycenter), ycenter + 1.2(yhi - ycenter)),
        label="Real function",
        legend = :outerbottom,
        title="max err = $(round(emax,digits=3))\nsum err^2 = $(round(esq,digits=3))",
    )
    
    scatter!(
        nodes,
        f.(nodes),
        label="Nodes (n=$n, $spacing)",
    )
    
    plot!(
        xs,
        P.(xs),
        label="Polynomial ($method)",
        
    )
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["method"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text : key")))], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}")))], Dict{Symbol, Any}(:attributes => ("data-bind" => "foreach : options_js"), :className => "buttons has-addons is-centered"))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("options_js" => (Observable{Vector{OrderedDict}} with 1 listeners. Value:
OrderedDict[OrderedDict{String, Any}("key" => "lagrange", "val" => 1, "id" => "idnt2XylYr"), OrderedDict{String, Any}("key" => "newton", "val" => 2, "id" => "id1ZvjFW39")], nothing), "index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/carbon/.julia/packages/Knockout/3gjS1/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/carbon/.julia/packages/Knockout/3gjS1/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/carbon/.julia/packages/InteractBase/h8Fwe/src/../assets/all.js"), Asset("css", nothing, "/Users/carbon/.julia/packages/InteractBase/h8Fwe/src/../assets/style.css"), Asset("css", nothing, "/Users/carbon/.julia/packages/Interact/pVyN2/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("options_js" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000016be67490, Task (runnable) @0x000000016be67490), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_

# Tabelki

In [8]:
function trial(n, params)
    dferr = DataFrame(n = [])
    [insertcols!(dferr, Symbol("err$i")=>[]) for i=eachindex(params)]
    for i = n
        row = Real[i]
        for (spacing, method) = params
            nodes = if spacing == :czeb
                         czeb.(1:i, i, xlo, xhi)
                     else
                         range(xlo, xhi, length=i)
                     end

            P = method(zip(nodes, f.(nodes)))
            err = sqerr(f, P, xs)
            push!(row, err)
        end
        push!(dferr, row)
    end
    dferr
end

trial (generic function with 1 method)

## Lagrange vs Newton
Newton umiera dla $n=40$.

In [9]:
trial(10:5:50, [(:czeb, lagrange), (:czeb, newton)])

,n,err1,err2
,Any,Any,Any
1,10,2882.85,2882.85
2,15,1276.07,1276.07
3,20,208.648,208.648
4,25,93.2554,93.2554
5,30,7.787,7.787
6,35,3.00158,3.00165
7,40,0.326033,1.78393
8,45,0.0993629,3557.34
9,50,0.00652393,7.84057e6


# Lagrange, duże $n$
Działa, dla większych $n$ za długo się liczy.

In [10]:
trial(100:100:1000, [(:czeb, lagrange)])

,n,err1
,Any,Any
1,100,1.06439e-12
2,200,6.18715e-26
3,300,8.35589e-26
4,400,1.48581e-25
5,500,2.14598e-25
6,600,5.50541e-25
7,700,4.80729e-25
8,800,6.9619e-25
9,900,6.82058e-25
